<a href="https://colab.research.google.com/github/octopus2023-inc/gensphere/blob/main/examples/agentic_workflows_from_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic workflows from prompts

This notebook will demonstrate how to create agentic workflows from high-level user prompts. GenSphere will process the prompt and create an execution graph of LLM API calls and function calls.   It will also attach tools for function calling and schemas for structured outputs along the way.

**Important:** The project uses openAI's o1-preview to generate the agentic workflow. o1-preview is available only to certain tiers, please check openAI documentation [here](https://help.openai.com/en/articles/9824962-openai-o1-preview-and-o1-mini-usage-limits-on-chatgpt-and-the-api)

Start by installing gensphere and importing modules

In [ ]:
!pip install gensphere

In [ ]:
from gensphere.hub import Hub
from gensphere.genflow import GenFlow
from gensphere.visualizer import Visualizer
import os
import logging
logging.getLogger('composio').setLevel(logging.WARNING)

In [ ]:
os.environ['OPENAI_API_KEY']="PLACE-YOUR-OPENAI-API-KEY-HERE" #Notice that this example will use o1-preview, which is not available to all user Tiers.

Now, we will pull from the Hub a GenSphere project, which takes as input a `task.txt` file and outputs a GenSphere project that accomplishes that task.The yml, functions and schemas file are saved locally after the flow execution completes as `text_to_workflow_result_yaml.yaml`, `text_to_workflow_result_functions.py`, `text_to_workflow_result_schemas.py` respectively.

In [ ]:
hub=Hub()
hub.pull(push_id='ddd4bfda-a33c-48e7-810c-4a1768973b9a',
         yaml_filename='yaml_file.yaml',
         functions_filename='gensphere_functions.py',
         schema_filename='gensphere_schemas.py',
         save_to_disk=True)

The files `yaml_file.yaml`, `gensphere_functions.py`, `gensphere_schemas.py` have been pulled from the hub saved locally.

You can visualize nodes in this GenSphere project with the Visualizer class, but we won't do it for now.

Now, let's execute the project by defining a task.

In [ ]:
task = """Your task is to generate script for 10 YouTube videos, about 5 minutes long each.
Our aim is to generate content for YouTube in an ethical way, while also ensuring we will go viral.
You should discover which are the topics with the highest chance of going viral today by searching the web.
Divide this search into multiple granular steps to get the best out of it. You can use Tavily and Firecrawl_scrape
to search the web and scrape URL contents, respectively. Then you should think about how to present these topics in order to make the video go viral.
Your script should contain detailed text (which will be passed to a text-to-speech model for voiceover),
as well as visual elements which will be passed to as prompts to image AI models like MidJourney.
You have full autonomy to create highly viral videos following the guidelines above.
Be creative and make sure you have a winning strategy."""


with open("task.txt", "w") as text_file:
    text_file.write(task)

flow=GenFlow('yaml_file.yaml','gensphere_functions.py','gensphere_schemas.py')
flow.parse_yaml()
flow.run()

Your agentic workflow was created, and its associated files were saved locally as `text_to_workflow_result_yaml.yaml`, `text_to_workflow_result_functions.py`, `text_to_workflow_result_schemas.py`. You can visualize your project with the Visualizer class:

In [ ]:
Visualizer('text_to_workflow_result_yaml.yaml',
           'text_to_workflow_result_functions.py',
           'text_to_workflow_result_schemas.py',
           address='127.0.0.1', port=8050).start_visualization()

Now that our agentic workflow is ready, we can execute it. We will need to add a few more api keys.

In [ ]:
os.environ['COMPOSIO_API_KEY']="place-your-api-key" #if you don't have one, visit composio.dev
os.environ['FIRECRAWL_API_KEY']="place-your-api-key" # if you don't have one, visit firecrawl.dev
os.environ['TAVILY_API_KEY']="place-your-api-key" # if you don't have one, visit tavily

In [ ]:
!composio add firecrawl #for this project, we will be using firecrawl. Get an API key and add it by following the steps here.

In [ ]:
!composio add tavily #for this project, we will be using tavily. Get an API key and add it by following the steps here.

In [ ]:
flow=GenFlow('text_to_workflow_result_yaml.yaml',
           'text_to_workflow_result_functions.py',
           'text_to_workflow_result_schemas.py')
flow.parse_yaml()
flow.run()

You can now access outputs with `flow.outputs` (which returns a dictionary, where keys are node names and values are also dictionaries, with one key for each node output as defined in the yaml file `text_to_workflow_result_yaml`).

In [ ]:
flow.outputs

By looking at your worklflow's graph, you can access final node outputs. As each call to openAI's API may return different results, check the visualization above to understand exactly how your project was created and what was the final node. This will allow you to adapt the code block below accordingly.

In [ ]:
#Print final outputs. You should adapt this according to how your project was created. Check the visualization above.
for i in flow.outputs.get('compile_final_output').get('final_output'):
  script=i.get('script')
  visual_prompt=i.get('visual_prompt')
  print(f'script: {script}')
  print(f'visual prompt: {visual_prompt}')